In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def dropna(x):
    return list(filter(lambda x: x > -np.inf, x))

path = '../data/VehicleID_152851_DriverID_22209/VehicleID_152851_DriverID_22209/File_ID_1'
data = pd.read_csv(path, na_values=" ")

## To do:

* Separate stops from cruising
* Make this detection more flexible
* Test on all other csvs
    * COROLLARY: upload own csvs to github
* Google API calls

## Ideas

### Known
* Speed for both speed measurements provided are kph
* Acceleration from GPS provided in gs (9.8 m/s^2)
* Cruising can be defined as staying within a certain margin of your recent average speed.
    * We will assume (but allow for adjustment on) a 5 kph margin in either direction
    * Average will be taken over the last 10/100 datapoints for GPS and network speed respectively
    * Will keep extra 10/100 datapoints at either end of the cruise, to look at interesting behavior


### Columns to use

* vtti.speed_gps
* vtti.speed_network
* vtti.pedal_brake_state
* vtti.pedal_gas_state
* vtti.engine_rpm
* vtti.accel (\_x, \_y, \_z)



* **vtti.video_frame**
    * Always necessary

In [ ]:
def get_rolling_average(series, n):
    avgs = list()
    for i in range(1, n+1):
        avgs.append(np.mean(series[:i]))
    for i in range(n, len(series)):
        avgs.append(np.mean(series[i-n:i]))
    return avgs

def identify_cruising(series, n, margin, p=False, display=False):
    averages = get_rolling_average(dropna(series), n)
    
    chunks = list()
    curr_avg = -np.inf
    curr_start = 0
    curr_n = 0
    
    for i in range(len(averages)):
        if averages[i] < curr_avg + margin and averages[i] > curr_avg - margin:
            curr_avg = (curr_avg * (curr_n + 1) + averages[i]) / (curr_n + 2)
            curr_n += 1
        else:
            if curr_n > n:
                # Add the chunk, as it is significant
                chunks.append(((curr_start, i), curr_avg))
            curr_avg = averages[i]
            curr_n = 0
            curr_start = i
    
    if p: print_cruising_data(chunks)
    if display: display_cruising_data(dropna(series), chunks)
    
    return chunks

def print_cruising_data(cruising_data):
    print("The following cruising segments were found:")
    for item in cruising_data:
        print("\tFrame {} to {}, with a mean speed of {:0.02f}kph".format(item[0][0], item[0][1], item[1]))

def display_cruising_data(series, chunks):
    for chunk in chunks:
        plt.plot(series[chunk[0][0]:chunk[0][1]])
        plt.title('Speed by frame for identified cruising chunk')
        plt.ylabel('kph')
        plt.xlabel('frame')
        plt.show()

In [ ]:
chunk_info = identify_cruising(data['vtti.speed_network'], 100, 2.5, True, True)

In [ ]:
plt.plot(dropna(data['vtti.speed_gps']))
plt.title('Speed (GPS)')
plt.show()

In [ ]:
plt.plot(dropna(data['vtti.speed_network']))
plt.title('Speed (network)')
plt.show()

In [ ]:
plt.plot(dropna(data['vtti.pedal_brake_state']))
plt.title('Brake state')
plt.show()

In [ ]:
plt.plot(dropna(data['vtti.pedal_gas_position']))
plt.title('Gas pedal position')
plt.show()

In [ ]:
plt.plot(dropna(data['vtti.engine_rpm_instant']))
plt.title('Engine RPM')
plt.show()

In [ ]:
accel_x = dropna(data['vtti.accel_x'])
accel_y = dropna(data['vtti.accel_y']) 
accel_z = dropna(data['vtti.accel_z']) 

plt.plot(accel_x)
plt.title('Acceleration in the X dimension')
plt.show()

plt.plot(accel_y)
plt.title('Acceleration in the Y dimension')
plt.show()

plt.plot(accel_z)
plt.title('Acceleration in the Z dimension')
plt.show()

In [ ]:
accel = list()
for i in range(len(accel_x)):
    accel.append((accel_x[i] ** 2 + accel_y[i] ** 2)  ** .5)
plt.plot(accel)
plt.title('Acceleration magnitude in XY plane at each observation')
plt.show()

In [ ]:
plt.plot(dropna((data['vtti.engine_rpm_instant'] - 400)/9000), alpha = 0.8)
plt.plot(np.abs(dropna(data['vtti.accel_x'])), c='g', alpha = .3)
plt.legend(['RPM', 'Acceleration in XY plane'])
plt.title('RPM vs XY Acceleration')
plt.show()